<a href="https://colab.research.google.com/github/jyotidabass/Medical_text_classification_five_class/blob/main/Medical_text_five_class_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Medical Text Classification using e-kNN 
where 'e' is epsilon & 'e' is the minimum similarity value requires to be in the nearest neighbour.

In [1]:
!git clone https://github.com/chaitanyakasaraneni/medicaltextclassification.git

Cloning into 'medicaltextclassification'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.


#### Importing the required libraries

In [2]:
#Importing the required libraries
import re
import math
import numpy as np
import pandas as pd
import string
import scipy as sp
import nltk
import time
import operator
from scipy import *
from scipy.sparse import *
from collections import defaultdict
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
#remove words less than length of 4
def filterLen(docs, minlen):
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]

#### Functions for building bag of words and csr_matrix ussing the bag of words

In [4]:
#Building Bag of words
def word_bag(docs):
#     nrows = len(docs)
    idx = {}
    tid = 0
    for d in docs:
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    return idx
    
#Building the sparse matrix
from collections import Counter
from scipy.sparse import csr_matrix
def build_matrix(docs, idx):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            #print(keys)
            if(k in idx):
                
                ind[j+n] = idx[k]
                val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

#### Functions to normalize the csr_matrix

In [5]:
# scale matrix and normalize its rows
def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

def csr_l2normalize(mat, copy=False, **kargs):
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

#### e-kNN Function

In [6]:
#kNN Function
class knn_main():
    def __init__(self):
        pass
    
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
    
    def predict(self, x_test,k,eps):
        self.x_test = x_test
        y_predict = []
        print(x_test.shape[0])
        for i,test_case in enumerate(x_test):
            temp = self.compute_distances(test_case, self.x_train)
            c = temp.tocoo(copy=True)
            d = self.sort_coo(c)
       #print(d[0])
            t = d[0:k]
        #print(t)
            dict_l ={}
        #print(k)
            for z,x in enumerate(t):
                index = x[1]
                similarity = x[2]
                label_t = self.y_train[index]
                if(z>0):
                    
                    if similarity > eps:
                        if label_t not in dict_l:
                            dict_l[label_t]=1
                        else:
                            dict_l[label_t]+=1
                else:
                    dict_l[label_t] =1
                
            m = max(dict_l.items(),key=operator.itemgetter(1))[0]
            y_predict.append(m)
            print("test case:", i+1,"Predicted :", m)
        return y_predict
    def sort_coo(self,m):
        tuples = zip(m.row, m.col, m.data)
        return sorted(tuples, key=lambda x: (x[2]),reverse=True)
    
    def compute_distances(self, X_test,train):        
        dot_pro = np.dot(X_test, train.T)
        return(dot_pro)

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### Reading the train data and storing it into list for performing classification

In [10]:
#Seperation of labels and text data
labels = []
texts = []
with open("/content/medicaltextclassification/train.dat", "r") as fh:
    train_lines = fh.readlines() 
for line in train_lines:
    splitline = line.split('\t')
    labels.append(splitline[0])
#     ps = word_tokenize()
    texts.append(nltk.word_tokenize(splitline[1].lower()))

len(texts)
# docs1 = filterLen(texts, 4)
# docs2 = filterLen(tex,4)

14438

#### Building word of bags for train data, building the csr_matrix and normalising it

In [11]:
train_text =  filterLen(texts, 4)
wordbag = word_bag(train_text)
train_mat = build_matrix(train_text,wordbag)
mat2 = csr_idf(train_mat, copy=True)
norm_train_mat = csr_l2normalize(mat2, copy=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

#### Train test spilt the train data to find the evaluate the performance of our code and then build csr_matrix for test data using the wordbag of the train data

In [12]:
X_train, X_test, y_train,y_test = train_test_split(texts,labels,test_size = 0.3)
test_mat = build_matrix(X_test,wordbag)
mat3 = csr_idf(test_mat, copy=True)
norm_test_mat = csr_l2normalize(mat3, copy=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

#### Calling the classifier function and performing predictions for the test split of train data

In [13]:
classifier = knn_main()
classifier.fit(norm_train_mat,labels)
g = classifier.predict(norm_test_mat,100,0.1)

4332
test case: 1 Predicted : 5
test case: 2 Predicted : 1
test case: 3 Predicted : 5
test case: 4 Predicted : 4
test case: 5 Predicted : 1
test case: 6 Predicted : 3
test case: 7 Predicted : 4
test case: 8 Predicted : 1
test case: 9 Predicted : 5
test case: 10 Predicted : 4
test case: 11 Predicted : 1
test case: 12 Predicted : 5
test case: 13 Predicted : 3
test case: 14 Predicted : 4
test case: 15 Predicted : 5
test case: 16 Predicted : 4
test case: 17 Predicted : 4
test case: 18 Predicted : 5
test case: 19 Predicted : 5
test case: 20 Predicted : 4
test case: 21 Predicted : 1
test case: 22 Predicted : 4
test case: 23 Predicted : 5
test case: 24 Predicted : 1
test case: 25 Predicted : 4
test case: 26 Predicted : 5
test case: 27 Predicted : 3
test case: 28 Predicted : 3
test case: 29 Predicted : 5
test case: 30 Predicted : 3
test case: 31 Predicted : 1
test case: 32 Predicted : 5
test case: 33 Predicted : 4
test case: 34 Predicted : 5
test case: 35 Predicted : 4
test case: 36 Predicted 

#### Building the confusion matrix and printing the performance report

In [14]:
cm=confusion_matrix(y_test ,g)
print(cm)
print(classification_report(y_test, g))

[[791  31  25  35  87]
 [ 31 250   6  23  98]
 [ 53  10 285  58 191]
 [ 22   8  24 765 109]
 [193 119  94 285 739]]
              precision    recall  f1-score   support

           1       0.73      0.82      0.77       969
           2       0.60      0.61      0.61       408
           3       0.66      0.48      0.55       597
           4       0.66      0.82      0.73       928
           5       0.60      0.52      0.56      1430

    accuracy                           0.65      4332
   macro avg       0.65      0.65      0.64      4332
weighted avg       0.65      0.65      0.65      4332



#### Now performing the above steps on the test data file

In [15]:
tst_tex = []
with open("/content/medicaltextclassification/test.dat", "r") as fr:
    test_lines = fr.readlines() 
for line in test_lines:
#         splitline = line.split()
    tst_tex.append(nltk.word_tokenize(line.lower()))
test_text = filterLen(tst_tex, 4)

In [16]:
test_mat = build_matrix(test_text,wordbag)
mat3 = csr_idf(test_mat, copy=True)
norm_test_mat = csr_l2normalize(mat3, copy=True)
# norm_test_mat

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecat

In [17]:
classifier = knn_main()
classifier.fit(norm_train_mat,labels)
#print(mat5)

In [18]:
g = classifier.predict(norm_test_mat,100,0.1)

Streaming output truncated to the last 5000 lines.
test case: 9443 Predicted : 4
test case: 9444 Predicted : 1
test case: 9445 Predicted : 4
test case: 9446 Predicted : 1
test case: 9447 Predicted : 2
test case: 9448 Predicted : 5
test case: 9449 Predicted : 1
test case: 9450 Predicted : 3
test case: 9451 Predicted : 5
test case: 9452 Predicted : 5
test case: 9453 Predicted : 5
test case: 9454 Predicted : 3
test case: 9455 Predicted : 4
test case: 9456 Predicted : 4
test case: 9457 Predicted : 5
test case: 9458 Predicted : 3
test case: 9459 Predicted : 4
test case: 9460 Predicted : 1
test case: 9461 Predicted : 5
test case: 9462 Predicted : 1
test case: 9463 Predicted : 4
test case: 9464 Predicted : 5
test case: 9465 Predicted : 5
test case: 9466 Predicted : 4
test case: 9467 Predicted : 5
test case: 9468 Predicted : 5
test case: 9469 Predicted : 1
test case: 9470 Predicted : 1
test case: 9471 Predicted : 3
test case: 9472 Predicted : 1
test case: 9473 Predicted : 5
test case: 9474 Pre

#### Writing the predicted class labels into a file

In [19]:
with open('program.dat', 'w') as f:
        for cls in g:
            f.write("%s\n" % cls)